In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt

In [8]:
df = pd.read_csv('schedulenets.csv')

In [11]:
df['time'] = pd.to_datetime(df['updatedAt'])

df['Date'] = df['time'].dt.date
df['Time'] = df['time'].dt.strftime('%H:%M')
df['Voltage']=df['block_1']

df = df[[ 'Date', 'Time','Voltage']]
df

,Date,Time,Voltage
1,2018-11-05,12:53,372.000000
9,2018-11-05,17:42,371.686299
0,2018-11-06,17:31,372.000000
14,2018-11-10,16:59,373.000000
3,2018-11-11,16:58,373.000000
...,...,...,...
10534,2021-06-11,09:30,550.000000
10538,2021-06-11,09:30,200.000000
10539,2021-06-11,04:27,546.000000
10540,2021-06-11,06:07,466.000000


In [12]:
df['Time_Intervals'] = df['Time'].apply(lambda x: f"{x[:2]}:{int(x[3:]) // 15 * 15:02d} to {x[:2]}:{(int(x[3:]) // 15 + 1) * 15:02d}")
df['Average_Voltage'] = df.groupby('Time_Intervals')['Voltage'].transform('mean')

df = df.groupby(['Date', 'Time_Intervals']).agg({'Average_Voltage': 'mean'}).reset_index()

data = df[[ 'Date', 'Time_Intervals','Average_Voltage']]
# df.to_csv('voltage_with_Intervals.csv', index=False) 
data

,Date,Time_Intervals,Average_Voltage
0,2018-11-05,12:45 to 12:60,417.272665
1,2018-11-05,17:30 to 17:45,407.169412
2,2018-11-06,17:30 to 17:45,407.169412
3,2018-11-10,16:45 to 16:60,386.672865
4,2018-11-11,16:45 to 16:60,386.672865
...,...,...,...
6540,2021-06-11,04:15 to 04:30,391.285290
6541,2021-06-11,06:00 to 06:15,401.741830
6542,2021-06-11,08:30 to 08:45,401.910695
6543,2021-06-11,09:00 to 09:15,397.025748


In [27]:
# Assuming your data is stored in a DataFrame named 'data'
data['Date'] = pd.to_datetime(data['Date'])

In [28]:
# Extract hours and minutes from 'Time_Intervals'
time_intervals = data['Time_Intervals'].str.extract(r'(\d+):(\d+) to (\d+):(\d+)')
data['Hour_Start'] = time_intervals[0].astype(int)
data['Minute_Start'] = time_intervals[1].astype(int)
data['Hour_End'] = time_intervals[2].astype(int)
data['Minute_End'] = time_intervals[3].astype(int)

# Convert 'Time_Start' and 'Time_End' to timedelta
data['Time_Start'] = pd.to_timedelta(data['Hour_Start'], unit='h') + pd.to_timedelta(data['Minute_Start'], unit='m')
data['Time_End'] = pd.to_timedelta(data['Hour_End'], unit='h') + pd.to_timedelta(data['Minute_End'], unit='m')

# Calculate the start and end times
data['Time_Interval_Start'] = data['Date'] + data['Time_Start']
data['Time_Interval_End'] = data['Date'] + data['Time_End']
data['Time_Interval_End']
data['Time_Interval_End']

0      2018-11-05 13:00:00
1      2018-11-05 17:45:00
2      2018-11-06 17:45:00
3      2018-11-10 17:00:00
4      2018-11-11 17:00:00
               ...        
6540   2021-06-11 04:30:00
6541   2021-06-11 06:15:00
6542   2021-06-11 08:45:00
6543   2021-06-11 09:15:00
6544   2021-06-11 09:45:00
Name: Time_Interval_End, Length: 6545, dtype: datetime64[ns]

In [29]:
# Feature engineering: Extract relevant time features
data['Hour'] = data['Time_Interval_Start'].dt.hour
data['Minute'] = data['Time_Interval_Start'].dt.minute

In [30]:
# Assuming 'Average_Voltage' is your target variable
target = 'Average_Voltage'

In [31]:
# Select features
features = ['Hour', 'Minute']  # You can add more features based on your data

In [32]:
# Split the data into train and test sets
train, test = train_test_split(data, test_size=0.2, shuffle=False) 

In [33]:
# Create XGBoost model
model = XGBRegressor()
model.fit(train[features], train[target])

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [34]:
# Make predictions on the test set
predictions = model.predict(test[features])

In [36]:
# Evaluate the model
rmse = sqrt(mean_squared_error(test[target], predictions))
print(f"Root Mean Squared Error (RMSE): {rmse}")

Root Mean Squared Error (RMSE): 0.0010426967621367902


In [38]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# rmse=rmse(predictions,test['y'])
# Calculate the mean of the test data
mean_value = test[target].mean()

# Calculate the accuracy in percentage
accuracy = (1 - (rmse / mean_value)) * 100

print(f"Accuracy: {accuracy:.2f}%")

Accuracy: 100.00%
